In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import jieba
import re
import os
import time
import gc

from sklearn.model_selection import train_test_split, KFold
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, BertModel, BertForSequenceClassification
from tqdm import tqdm
from sklearn.metrics import f1_score, auc, roc_curve

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [2]:
from config import root_path, stopwords_path, train_query_path, train_reply_path, device, best_model_path, batch_size, \
    test_query_path, test_reply_path, user_dict_path, train_path, train_all_path, test_path, dev_path, pretrained_bert_path, lr, is_cuda, max_gradient_norm

## preprocessor

In [3]:
def load_stopwords(file):
    """
    加载停用词
    :param file:
    :return:
    """
    with open(file, 'r', encoding='utf-8') as fp:
        stopwords = fp.read().strip().splitlines()
    return stopwords

def filter_stopwords(sentence, stopwords):
    """
    过滤停用词
    :param sentence:
    :param stopwords:
    :return:
    """
    text = ' '.join([word for word in sentence.split() if word not in stopwords])
    return text

def seg_sentence(sentence):
    """
    分词
    :param sentence:
    :return:
    """
    segeds = jieba.cut(sentence.strip())
    return ' '.join(segeds)

def filter_content(sentence):
    """
    过滤特定内容
    :param sentence:
    :return:
    """
    # 替换数字，ccks数据中包含阿拉伯数字、中文数字
    sentence = re.sub(r"([1-9]\d*\.?\d*)|(0\.\d*[1-9])", "数字x", sentence)
    sentence = re.sub(r"([\u96f6\u4e00\u4e8c\u4e09\u56db\u4e94\u516d\u4e03\u516b\u4e5d\u5341\u767e\u5343\u4e07]+)", "数字x", sentence)

    return sentence

def generate_dataset(query_path, reply_path, to_path, stopwords, dev_path=None, mode='train'):
    """
    生成数据集
    :param query_path:
    :param reply_path:
    :param to_path: 保存训练集/测试集的路径
    :param mode:train / test / dev
    :return:
    """
    if mode in ['train', 'dev']:
        names = ['dialog_id', 'reply_id', 'reply_content', 'label']
        encoding = 'utf-8'
    else:
        names = ['dialog_id', 'reply_id', 'reply_content']
        encoding = 'gbk'

    query_df = pd.read_csv(query_path, sep='\t', encoding=encoding, names=['dialog_id', 'question'])

    reply_df = pd.read_csv(reply_path, sep='\t', encoding=encoding, names=names)
    query_df = query_df.dropna()
    reply_df = reply_df.dropna()

    df = pd.merge(query_df, reply_df, how='inner')

    for col in ['question', 'reply_content']:
        # 过滤特殊文本
        df[col] = df[col].apply(lambda x: filter_content(x))
        # 分词
        df[col] = df[col].apply(lambda x: seg_sentence(x))
        # 过滤停用词（仅过滤标点符号）
        df[col] = df[col].apply(lambda x: filter_stopwords(x, stopwords))
        # 过滤停用词（仅过滤标点符号）后，会出现空字符串（原字符串仅有标点符号，过滤后就为空）的情况
        df[col].replace(to_replace=r'^\s*$', value=np.nan, inplace=True, regex=True)
        df[col].fillna('符号x', inplace=True)
    if mode == 'dev':
        # 如果是dev，则将train分为训练集和验证集
        train, dev = train_test_split(df, test_size=0.25)
        train.to_csv(to_path, index=0)
        dev.to_csv(dev_path, index=0)
    else:
        df.to_csv(to_path, index=0)

In [4]:
jieba.load_userdict(user_dict_path)
stopwords = load_stopwords(stopwords_path)
generate_dataset(train_query_path, train_reply_path, train_all_path, stopwords)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.925 seconds.
Prefix dict has been built succesfully.


## dataset

In [3]:
class QAMatchDataset(Dataset):
    def __init__(self, df, tokenizer, mode):
        assert mode in ['train', 'dev', 'test']

        self.mode = mode
        self.tokenizer = tokenizer
        self.df = df
        # self.df = pd.read_csv(file)
        # self.seqs, self.seq_masks, self.seq_segments, self.labels = self.get_input(file)

    def __getitem__(self, idx):
        token_seq_1 = self.df.iloc[idx]['question']
        token_seq_2 = self.df.iloc[idx]['reply_content']
        if self.mode in ['train', 'dev']:
            label_tensor = torch.tensor(self.df.iloc[idx]['label'])
        else:
            label_tensor = None
        token_seq_1 = self.tokenizer.tokenize(token_seq_1)
        token_seq_2 = self.tokenizer.tokenize(token_seq_2)

        seq = ["[CLS]"] + token_seq_1 + ["[SEP]"] + token_seq_2 + ["[SEP]"]
        seq = self.tokenizer.convert_tokens_to_ids(seq)

        seq_segments = [0] * (len(token_seq_1) + 2) + [1] * (len(token_seq_2) + 1)

        return torch.Tensor(seq).type(torch.long), torch.Tensor(seq_segments).type(torch.long), label_tensor

    def collate_fn(self, samples):
        seqs = [s[0] for s in samples]
        seq_segments = [s[1] for s in samples]

        if self.mode in ['train', 'dev']:
            labels = torch.stack([s[2] for s in samples])
        else:
            labels = None

        seqs = pad_sequence(seqs, batch_first=True)
        seq_segments = pad_sequence(seq_segments, batch_first=True)

        # attention mask处理
        seq_masks = torch.zeros(seqs.shape, dtype=torch.long)
        seq_masks = seq_masks.masked_fill(seqs != 0, 1)

        return seqs, seq_masks, seq_segments, labels

    def __len__(self):
        return len(self.df)

## model

In [4]:
class BertModelTrain(nn.Module):
    def __init__(self):
        super(BertModelTrain, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained(pretrained_bert_path, num_labels=2)
        self.device = device
        for param in self.bert.parameters():
            param.requires_grad = True     # fine-tune，每个参数都要更新

    def forward(self, batch_seqs, batch_seq_masks, batch_seq_segments, labels=None):
        """
        :param batch_seqs: input_ids
        :param batch_seq_masks: attention_mask
        :param batch_seq_segments: token_type_ids
        :param labels:
        :return: outputs: (loss, logits, ...)
                 outputs: (logits, ...)
        """
        outputs = self.bert(input_ids=batch_seqs,
                            attention_mask=batch_seq_masks,
                            token_type_ids=batch_seq_segments,
                            labels=labels)

        if labels is not None:
            logits = outputs[1]                   # shape: (batch, 2)
            probabilities = nn.functional.softmax(logits, dim=-1)
            # 将label:[0,1,1,0] -> y_true: tensor[[1,0], [0,1], [0,1], [1,0]], 即在每个label上的真实概率
            y_true = torch.zeros(logits.shape)    # shape: (batch, 2)
            for i in range(y_true.shape[0]):
                y_true[i, labels[i]] = 1
            loss = nn.functional.binary_cross_entropy_with_logits(logits.cpu(), y_true)
            # loss = outputs[0]
            outputs = (loss, ) + (logits,) + (probabilities, )
        # probabilities = nn.functional.softmax(logits, dim=-1)
        else:
            logits = outputs[0]
            probabilities = nn.functional.softmax(logits, dim=-1)
            outputs = (logits, probabilities)
        return outputs

## train

In [5]:
def train(train_dataloader, dev_dataloader, bert_tokenizer, best_model_path, fold, epochs=5, patience=3, checkpoint=None):
    # ---------------------- Model definition ---------------------- #
    print("\t* Building model...")
    bulid_time = time.time()
    model = BertModelTrain().to(device)
    print("\t* Building model time:{:.4f}s".format(time.time()-bulid_time))
    # ---------------------- Preparation for training -------------- #
    param_optimizer = list(model.named_parameters())
    # 这里，指定部分参数不参与权重衰减。
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [{
        'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay': 0.001
    }, {
        'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay': 0.0
    }]
#     optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    optimizer = Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.85, patience=patience)

    best_score = 0.0    # 记录validation最好的结果
    best_thres = 0.0
    start_epoch = 1
    # Data for loss curves plot
    epoch_count = []
    train_losses = []
    valid_losses = []
    best_model_saved_path = os.path.join(best_model_path, 'best-fine-tune-V1.4-k1.bin')

    # 如果有给参数checkpoint，则继续训练
    if checkpoint:
        checkpoint = torch.load(checkpoint)
        start_epoch = checkpoint['epoch'] + 1
        best_score = checkpoint['best_score']
        best_thres = checkpoint['best_thres']
        print("\t* Training will continue on existing model from epoch{}...".format(start_epoch))
        model.bert.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch_count = checkpoint['epoch_count']
        train_losses = checkpoint['train_losses']
        valid_losses = checkpoint['valid_losses']

    # Compute loss and accuracy before starting (or resuming) training
    # 如果准备start training，这里的valid结果就是预训练BERT（做fine-tune之前）对下游任务的效果
    # 如果准备resuming training，这里的valid结果就是上一次fine-tune的结果
    valid_loss, valid_accuracy, valid_f1, valid_auc, thres = validate(model, dev_dataloader)
    print("\t* Validation loss before training: {:.4f}, accuracy:{:.4f}, "
          "f1_score: {:.4f}, best_thres: {:.4f}, auc: {:.4f}".
          format(valid_loss, (valid_accuracy * 100), valid_f1, thres, valid_auc))
    print("\n", 20 * "=", "Training Bert model o device: {}".format(device), 20 * "=")

    patience_counter = 0
    for epoch in range(start_epoch, epochs+1):
        print("-> Start epoch {}".format(epoch))
        epoch_count.append(epoch)
        # train
        epoch_time, epoch_loss, epoch_accuracy, epoch_f1, epoch_auc = train_for_one_epoch(model,
                                                                                          train_dataloader,
                                                                                          optimizer,
                                                                                          max_gradient_norm)
        train_losses.append(epoch_loss)
        print("-> Training time:{:.4f}s, loss: {:.4f}, accuracy: {:.4f}%, f1_score: {:.4f}, auc: {:.4f}".
              format(epoch_time, epoch_loss, epoch_accuracy*100, epoch_f1, epoch_auc))

        # validation
        valid_loss, valid_accuracy, valid_f1, valid_auc, thres = validate(model, dev_dataloader)
        print("-> Validation loss: {:.4f}, accuracy: {:.4f}%, f1_score: {:.4f}, best_thres: {:.4f}, auc: {:.4f}".
              format(valid_loss, valid_accuracy * 100, valid_f1, thres, valid_auc))
        valid_losses.append(valid_loss)
        scheduler.step(valid_auc)

        # 以valid_auc为评测标准
        
        if valid_auc < best_score:
            patience_counter += 1
        else:
            best_score = valid_auc
            best_thres = thres
            patience_counter = 0
            best_model_saved_path = os.path.join(best_model_path, 'best-fine-tune-V1.4-k'+str(fold)+'.bin')
            torch.save({
                "epoch": epoch,
                "model": model.bert.state_dict(),
                "best_score": best_score,
                "best_thres": best_thres,
                "epochs_count": epoch_count,
                "train_losses": train_losses,
                "valid_losses": valid_losses
            }, best_model_saved_path)

        if patience_counter >= patience:
            print("-> Early stopping: patience limit reached, stopping...")
            break
    if patience_counter != 0:
        # 如果最后一个epoch不是最好的模型，则读取之前的最好的模型
        best_checkpoint = torch.load(best_model_saved_path)
        model.bert.load_state_dict(best_checkpoint['model'])
    return model


def train_for_one_epoch(model, dataloader, optimizer, max_gradient_norm):
    model.train()

    epoch_start_time = time.time()
    running_loss = 0.0   # 记录整个epoch的累加loss
    correct_count = 0.0
    batch_avg_time = 0.0 # 记录该epoch平均batch花费时间
    all_preds = []
    all_pred_probas = []
    all_labels = []

    tqdm_dataloader = tqdm(dataloader)
    for batch_index, data in enumerate(tqdm_dataloader):
        batch_start_time = time.time()
        if is_cuda:
            data = [t.to(device) for t in data if t is not None]
        # 梯度置零
        optimizer.zero_grad()
        seqs, seq_masks, seq_segments, labels = data
        outputs = model(seqs, seq_masks, seq_segments, labels)
        # 回传梯度
        loss = outputs[0]
        logits = outputs[1]
        probabilities = outputs[2]
        # probabilities = nn.functional.softmax(logits, dim=-1)
        loss.backward()
        # 梯度裁剪
        nn.utils.clip_grad_norm_(model.parameters(), max_gradient_norm)
        optimizer.step()

        running_loss += loss.item()
        _, pred = torch.max(logits, dim=1)
        correct_count = correct_count + (pred == labels).sum().item()
        batch_avg_time += time.time() - batch_start_time
        # all_preds.append(pred)
        all_labels.append(labels.cpu())
        all_pred_probas.append(probabilities.detach().cpu())

        description = "Batch num: {}. Avg. batch proc. time: {:.4f}s, loss: {:.4f}".\
            format(batch_index+1, batch_avg_time/(batch_index+1), running_loss/(batch_index+1))
        tqdm_dataloader.set_description(description)
        del data
        torch.cuda.empty_cache()
        
    all_labels = torch.cat(all_labels)    # 把每个batch的labels平铺成一维tensor (samples, )
    # all_preds = torch.cat(all_preds)      # 把每个batch的preds平铺成一维tensor (samples, )
    all_pred_probas = torch.cat(all_pred_probas) # 把每个batch的probas平铺成tensor (samples, 2)

    fpr, tpr, thresholds = roc_curve(all_labels, all_pred_probas[:, 1], pos_label=1)

    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = correct_count / len(dataloader.dataset)
    epoch_time = time.time() - epoch_start_time
    epoch_auc = auc(fpr, tpr)
    # epoch_f1 = f1_score(all_labels, all_preds)

    # return epoch_time, epoch_loss, epoch_accuracy, epoch_f1, epoch_auc
    return epoch_time, epoch_loss, epoch_accuracy, 0, epoch_auc


def validate(model, dataloader):
    model.eval()
    running_loss = 0.0  # 记录整个epoch的累加loss
    correct_count = 0.0
    # all_preds = []
    all_labels = []
    all_pred_probas = []
    tqdm_dataloader = tqdm(dataloader)

    # Deactivate autograd for evaluation
    with torch.no_grad():   # 必须加这个，减少显存的使用
        for batch_index, data in enumerate(tqdm_dataloader):
            if is_cuda:
                data = [t.to(device) for t in data if t is not None]

            seqs, seq_masks, seq_segments, labels = data
            outputs = model(seqs, seq_masks, seq_segments, labels)
            loss = outputs[0]
            logits = outputs[1]
            probabilities = outputs[2]
            # probabilities = nn.functional.softmax(logits, dim=-1)

            running_loss += loss.item()
            # _, pred = torch.max(logits, dim=1)

            # correct_count = correct_count + (pred == labels).sum().item()
            # all_preds.append(pred.cpu())
            all_labels.append(labels.cpu())
            all_pred_probas.append(probabilities.cpu())
            
            del data
            torch.cuda.empty_cache()
                    
    all_labels = torch.cat(all_labels)  # 把每个batch的labels平铺成一维tensor shape: (samples, )
    # all_preds = torch.cat(all_preds)  # 把每个batch的preds平铺成一维tensor shape: (samples, )
    all_pred_probas = torch.cat(all_pred_probas)  # 把每个batch的probas变成tensor（原来是[tensor, tensor, ...]）


    best_f1, best_thres = search_f1(all_labels, all_pred_probas[:, 1])
    all_preds = (all_pred_probas[:, 1] > best_thres).type(torch.long)
    correct_count = (all_preds == all_labels).sum().item()

    fpr, tpr, thresholds = roc_curve(all_labels, all_pred_probas[:, 1], pos_label=1)

    valid_loss = running_loss / len(dataloader)
    valid_acc = correct_count / len(dataloader.dataset)
    # valid_f1 = f1_score(all_labels, all_preds)
    valid_f1 = best_f1
    valid_auc = auc(fpr, tpr)
    return valid_loss, valid_acc, valid_f1, valid_auc, best_thres
    # return valid_loss, valid_acc, 0, 0
    
def search_f1(y_true, y_pred):
    """

    :param y_true: 一维tensor
    :param y_pred: 一维tensor，y_pred[i]表示第i个样本在label为1上的预测概率
    :return:
    """
    best_score = 0.0
    best_thres = 0.0
    for i in range(30, 60):
        thres = i / 100
        y_pred_bin = (y_pred > thres)   # 大于thres的为1，小于thres的为0
        # print("y_pred_bin shape:", y_pred_bin.shape)
        score = f1_score(y_true, y_pred_bin)
        if score > best_score:
            best_score = score
            best_thres = thres

    return best_score, best_thres
    
def get_pred_probas(model, dataloader):
    model.eval()
    probas = None

    with torch.no_grad():
        for data in dataloader:
            # 将所有tensors移到GPU上
            if is_cuda:
                data = [t.to(device) for t in data if t is not None]

            seqs, seq_masks, seq_segments = data[:3]
            outputs = model(seqs,
                            seq_masks,
                            seq_segments)
            logits = outputs[0]
            probabilities = outputs[1]   # (batch, 2)
            # probabilities = nn.functional.softmax(logits, dim=1)
            # _, pred = torch.max(logits.data, dim=1)

            if probas is None:
                probas = probabilities
            else:
                # 将每个batch的预测结果拼接起来
                probas = torch.cat([probas, probabilities])
                
            del data
            torch.cuda.empty_cache()

    return probas.cpu()

## KFold

In [6]:
def k_fold_cross_val(train_df, test_df, k, bert_tokenizer, best_model_path, version):
    kf = KFold(n_splits=5)
    test_dataset = QAMatchDataset(test_df, bert_tokenizer, mode='test')
    test_dataloader = DataLoader(test_dataset,
                                 batch_size=batch_size,
                                 num_workers=3,
                                 collate_fn=test_dataset.collate_fn)
    dev_labels = []
    dev_probas = []
    test_probas = []
    for fold, (train_idxs, dev_idxs) in enumerate(kf.split(train_df)):
        print("\t* Start "+str(fold)+" fold")
        dev_labels.extend(train_df.iloc[dev_idxs]['label'].tolist())
        # ---------------------- Data loading -------------------------- #
        print("\t* Building dataset...")
        train_dataset = QAMatchDataset(train_df.iloc[train_idxs], bert_tokenizer, 'train')
        dev_dataset = QAMatchDataset(train_df.iloc[dev_idxs], bert_tokenizer, 'dev')

        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=3,
                                      collate_fn=train_dataset.collate_fn)
        dev_dataloader = DataLoader(dev_dataset, batch_size=batch_size, num_workers=3,
                                    collate_fn=dev_dataset.collate_fn)
        best_model_fold_path = os.path.join(best_model_path, 'best-fine-tune-'+version+'-k'+str(fold)+'.bin')
        if not(os.path.exists(best_model_fold_path)):
            # 若已有
            model = train(train_dataloader, dev_dataloader, bert_tokenizer, best_model_path, 
                          fold, epochs=5, patience=3, checkpoint=None)
        else:
            checkpoint = torch.load(best_model_fold_path)
            model = BertModelTrain().to(device)
            model.bert.load_state_dict(checkpoint['model'])
        
        dev_proba = get_pred_probas(model, dev_dataloader)
        test_proba = get_pred_probas(model, test_dataloader)
        
        dev_probas.append(dev_proba)  # (k, len(dev_idxs), 2)
        test_probas.append(test_proba) # (k, len(test_dataset), 2)
#         model.to(torch.device('cpu'))
        del model, train_dataloader, dev_dataloader, checkpoint
        torch.cuda.empty_cache() 
        time.sleep(5)
    
    dev_probas = torch.cat(dev_probas)  # (len(train_df), 2)    # 把每一折的验证集的预测结果拼接，得到整个训练集的预测结果
    f1, thres = search_f1(torch.tensor(dev_labels), dev_probas) # 找最好的F1和thres
    
    test_probas = torch.stack(test_probas) # (k, len(test_dataset), 2)， 只是把[tensor, tensor, ... ]转为tensor
    test_probas = torch.mean(test_probas, dim=0)  # (len(test_dataset), 2)  取每一折的平均
    test_preds = (test_probas[:, 1] > thres).type(torch.long)
    return test_preds

## 操作

In [7]:
version = 'V1.4'
train_df = pd.read_csv(train_all_path)
test_df = pd.read_csv(test_path)
k = 5
bert_tokenizer = BertTokenizer.from_pretrained(os.path.join(pretrained_bert_path, 'vocab.txt'))
print("\t* K fold training and validating...")
test_preds = k_fold_cross_val(train_df, test_df, k, bert_tokenizer, best_model_path, version)
print("\t*Predicting...")
test_df['pred'] = test_preds.cpu().numpy()
print("\t*Saving...")
time_str = '' + time.strftime("%Y%m%d%H%M", time.localtime())
test_df[['dialog_id', 'reply_id', 'pred']].to_csv(os.path.join(root_path,'submission/'+version+'_'+time_str+'.csv'),
                                                  sep='\t',
                                                  index=0,
                                                  header=0)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


	* K fold training and validating...
	* Start 0 fold
	* Building dataset...
	* Start 1 fold
	* Building dataset...
	* Building model...


  0%|          | 0/135 [00:00<?, ?it/s]

	* Building model time:4.0103s


  0%|          | 0/540 [00:00<?, ?it/s]

	* Validation loss before training: 0.6405, accuracy:42.9002, f1_score: 0.1520, best_thres: 0.3000, auc: 0.2824

 ==================== Training Bert model o device: cuda ====================
-> Start epoch 1


Batch num: 7. Avg. batch proc. time: 0.2611s, loss: 0.6465:   1%|▏         | 7/540 [00:02<02:58,  2.98it/s]

RuntimeError: CUDA out of memory. Tried to allocate 76.00 MiB (GPU 0; 11.19 GiB total capacity; 9.34 GiB already allocated; 95.86 MiB free; 330.73 MiB cached)